**This notebook is an exercise in the [Advanced SQL](https://www.kaggle.com/learn/advanced-sql) course.  You can reference the tutorial at [this link](https://www.kaggle.com/alexisbcook/joins-and-unions).**

---


# Introduction

Here, you'll use different types of SQL **JOINs** to answer questions about the [Stack Overflow](https://www.kaggle.com/stackoverflow/stackoverflow) dataset.

Before you get started, run the following cell to set everything up.

In [2]:
# Set up feedback system
from learntools.core import binder
binder.bind(globals())
from learntools.sql_advanced.ex1 import *
print("Setup Complete")

Using Kaggle's public dataset BigQuery integration.
Setup Complete


The code cell below fetches the `posts_questions` table from the `stackoverflow` dataset.  We also preview the first five rows of the table.

In [3]:
from google.cloud import bigquery

# Create a "Client" object
client = bigquery.Client()

# Construct a reference to the "stackoverflow" dataset
dataset_ref = client.dataset("stackoverflow", project="bigquery-public-data")

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

# Construct a reference to the "posts_questions" table
table_ref = dataset_ref.table("posts_questions")

# API request - fetch the table
table = client.get_table(table_ref)

# Preview the first five lines of the table
client.list_rows(table, max_results=5).to_dataframe()

Using Kaggle's public dataset BigQuery integration.


,id,title,body,accepted_answer_id,answer_count,comment_count,community_owned_date,creation_date,favorite_count,last_activity_date,last_edit_date,last_editor_display_name,last_editor_user_id,owner_display_name,owner_user_id,parent_id,post_type_id,score,tags,view_count
0,65070674,NewRelic APM cpu usage shows incorrect values ...,<p>Here goes charts of CPU usage of same pod. ...,None,0,0,None,2020-11-30 09:07:52.263000+00:00,None,2020-11-30 09:07:52.263000+00:00,None,None,None,None,1448545,None,1,0,newrelic|kubernetes-pod,1
1,65070689,NewRelic APM cpu usage shows incorrect values ...,<p>Here goes charts of CPU usage of same pod. ...,None,0,0,None,2020-11-30 09:09:16.030000+00:00,None,2020-11-30 09:09:16.030000+00:00,None,None,None,None,1448545,None,1,0,newrelic|kubernetes-pod,1
2,64972916,Gitlab : How to batch modify the visibility of...,<p>I had to change the visibility of many proj...,None,1,0,None,2020-11-23 17:05:43.130000+00:00,None,2020-11-23 17:05:43.130000+00:00,None,None,None,None,1416845,None,1,0,gitlab|batch-processing,2
3,65007914,Upload internal Android application on WmWare(...,<p>I am trying to upload Android debug build o...,None,0,0,None,2020-11-25 15:37:33.790000+00:00,None,2020-11-25 15:37:33.790000+00:00,None,None,None,None,8659884,None,1,0,android,2
4,65023562,swig: how to add context manager methods to op...,"<p>when swigging opaque handles (in my case, p...",None,1,0,None,2020-11-26 14:03:29.567000+00:00,None,2020-11-26 14:03:29.567000+00:00,None,None,None,None,1368566,None,1,0,python|swig,2


We also take a look at the `posts_answers` table.

In [4]:
# Construct a reference to the "posts_answers" table
table_ref = dataset_ref.table("posts_answers")

# API request - fetch the table
table = client.get_table(table_ref)

# Preview the first five lines of the table
client.list_rows(table, max_results=5).to_dataframe()

,id,title,body,accepted_answer_id,answer_count,comment_count,community_owned_date,creation_date,favorite_count,last_activity_date,last_edit_date,last_editor_display_name,last_editor_user_id,owner_display_name,owner_user_id,parent_id,post_type_id,score,tags,view_count
0,14572177,None,<p>If you want to get All destop items you wil...,None,None,0,None,2013-01-28 22:11:00.227000+00:00,None,2013-01-28 22:11:00.227000+00:00,NaT,None,NaN,None,1849109,14572041,2,0,None,None
1,14572191,None,"<p>Look at this function it will work, I've co...",None,None,0,None,2013-01-28 22:12:33.897000+00:00,None,2013-01-28 22:23:02.473000+00:00,2013-01-28 22:23:02.473000+00:00,None,573032.0,None,573032,14557984,2,0,None,None
2,14572199,None,<p>Do you need to update the UI often or not?<...,None,None,0,None,2013-01-28 22:12:49.057000+00:00,None,2013-01-28 22:12:49.057000+00:00,NaT,None,NaN,None,1416035,14572118,2,0,None,None
3,14572208,None,<p>Look into using a Repeater or DataList cont...,None,None,0,None,2013-01-28 22:13:14.830000+00:00,None,2013-01-28 22:13:14.830000+00:00,NaT,None,NaN,None,358015,14572103,2,0,None,None
4,14572217,None,<p>There are some methods:</p>\n\n<ol>\n<li>Di...,None,None,0,None,2013-01-28 22:14:41.693000+00:00,None,2013-01-28 22:14:41.693000+00:00,NaT,None,NaN,None,448089,14572089,2,0,None,None


You will work with both of these tables to answer the questions below.

# Exercises

### 1) How long does it take for questions to receive answers?

You're interested in exploring the data to have a better understanding of how long it generally takes for questions to receive answers.  Armed with this knowledge, you plan to use this information to better design the order in which questions are presented to Stack Overflow users.

With this goal in mind, you write the query below, which focuses on questions asked in January 2018.  It returns a table with two columns:
- `q_id` - the ID of the question
- `time_to_answer` - how long it took (in seconds) for the question to receive an answer

Run the query below (without changes), and take a look at the output.

In [5]:
first_query = """
              SELECT q.id AS q_id,
                  MIN(TIMESTAMP_DIFF(a.creation_date, q.creation_date, SECOND)) as time_to_answer
              FROM `bigquery-public-data.stackoverflow.posts_questions` AS q
                  INNER JOIN `bigquery-public-data.stackoverflow.posts_answers` AS a
              ON q.id = a.parent_id
              WHERE q.creation_date >= '2018-01-01' and q.creation_date < '2018-02-01'
              GROUP BY q_id
              ORDER BY time_to_answer
              """

first_result = client.query(first_query).result().to_dataframe()
print("Percentage of answered questions: %s%%" % \
      (sum(first_result["time_to_answer"].notnull()) / len(first_result) * 100))
print("Number of questions:", len(first_result))
first_result.head()

Percentage of answered questions: 100.0%
Number of questions: 134577


,q_id,time_to_answer
0,48195422,0
1,48210143,0
2,48489235,0
3,48477364,0
4,48288278,0


You're surprised at the results and strongly suspect that something is wrong with your query.  In particular,
- According to the query, 100% of the questions from January 2018 received an answer.  But, you know that ~80% of the questions on the site usually receive an answer.
- The total number of questions is surprisingly low.  You expected to see at least 150,000 questions represented in the table.

Given these observations, you think that the type of **JOIN** you have chosen has inadvertently excluded unanswered questions.  Using the code cell below, can you figure out what type of **JOIN** to use to fix the problem so that the table includes unanswered questions?

**Note**: You need only amend the type of **JOIN** (i.e., **INNER**, **LEFT**, **RIGHT**, or **FULL**) to answer the question successfully.

In [6]:
# Your code here
correct_query = """
              SELECT q.id AS q_id,
                  MIN(TIMESTAMP_DIFF(a.creation_date, q.creation_date, SECOND)) as time_to_answer
              FROM `bigquery-public-data.stackoverflow.posts_questions` AS q
                  LEFT JOIN `bigquery-public-data.stackoverflow.posts_answers` AS a
              ON q.id = a.parent_id
              WHERE q.creation_date >= '2018-01-01' and q.creation_date < '2018-02-01'
              GROUP BY q_id
              ORDER BY time_to_answer
              """

# Check your answer
q_1.check()

# Run the query, and return a pandas DataFrame
correct_result = client.query(correct_query).result().to_dataframe()
print("Percentage of answered questions: %s%%" % \
      (sum(correct_result["time_to_answer"].notnull()) / len(correct_result) * 100))
print("Number of questions:", len(correct_result))

,q_id,time_to_answer
0,48167544,NaN
1,48073776,NaN
2,48481668,NaN
3,48540039,NaN
4,48539052,NaN


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

Percentage of answered questions: 82.89006873783538%
Number of questions: 162356


In [ ]:
# Lines below will give you a hint or solution code
#q_1.hint()
#q_1.solution()

### 2) Initial questions and answers, Part 1

You're interested in understanding the initial experiences that users typically have with the Stack Overflow website.  Is it more common for users to first ask questions or provide answers?  After signing up, how long does it take for users to first interact with the website?  To explore this further, you draft the (partial) query in the code cell below.

The query returns a table with three columns:
- `owner_user_id` - the user ID
- `q_creation_date` - the first time the user asked a question 
- `a_creation_date` - the first time the user contributed an answer 

You want to keep track of users who have asked questions, but have yet to provide answers.  And, your table should also include users who have answered questions, but have yet to pose their own questions.  

With this in mind, please fill in the appropriate **JOIN** (i.e., **INNER**, **LEFT**, **RIGHT**, or **FULL**) to return the correct information.  

**Note**: You need only fill in the appropriate **JOIN**.  All other parts of the query should be left as-is.  (You also don't need to write any additional code to run the query, since the `cbeck()` method will take care of this for you.)

To avoid returning too much data, we'll restrict our attention to questions and answers posed in January 2019.  We'll amend the timeframe in Part 2 of this question to be more realistic!

In [7]:
# Your code here
q_and_a_query = """
                SELECT q.owner_user_id AS owner_user_id,
                    MIN(q.creation_date) AS q_creation_date,
                    MIN(a.creation_date) AS a_creation_date
                FROM `bigquery-public-data.stackoverflow.posts_questions` AS q
                    FULL JOIN `bigquery-public-data.stackoverflow.posts_answers` AS a
                ON q.owner_user_id = a.owner_user_id 
                WHERE q.creation_date >= '2019-01-01' AND q.creation_date < '2019-02-01' 
                    AND a.creation_date >= '2019-01-01' AND a.creation_date < '2019-02-01'
                GROUP BY owner_user_id
                """

# Check your answer
q_2.check()

,owner_user_id,q_creation_date,a_creation_date
0,5061553,2019-01-28 13:45:17.927000+00:00,2019-01-01 09:18:19.020000+00:00
1,6848923,2019-01-16 11:03:27.257000+00:00,2019-01-01 09:22:09.940000+00:00
2,10832127,2019-01-22 19:31:52.437000+00:00,2019-01-01 06:08:41.347000+00:00
3,7585847,2019-01-01 02:51:24.567000+00:00,2019-01-01 10:59:28.247000+00:00
4,8010659,2019-01-24 12:19:34.663000+00:00,2019-01-01 11:20:54.323000+00:00


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [8]:
# Lines below will give you a hint or solution code
q_2.hint()
#q_2.solution()

<IPython.core.display.Javascript object>

<span style="color:#3366cc">Hint:</span> Which type of JOIN always returns all rows from both tables?

### 3) Initial questions and answers, Part 2

Now you'll address a more realistic (and complex!) scenario.  To answer this question, you'll need to pull information from *three* different tables!  This syntax very similar to the case when we have to join only two tables.  For instance, consider the three tables below.

![three tables](https://i.imgur.com/OyhYtD1.png)

We can use two different **JOINs** to link together information from all three tables, in a single query.

![double join](https://i.imgur.com/G6buS7P.png)

With this in mind, say you're interested in understanding users who joined the site in January 2019.  You want to track their activity on the site: when did they post their first questions and answers, if ever?

Write a query that returns the following columns:
- `id` - the IDs of all users who created Stack Overflow accounts in January 2019 (January 1, 2019, to January 31, 2019, inclusive)
- `q_creation_date` - the first time the user posted a question on the site; if the user has never posted a question, the value should be null
- `a_creation_date` - the first time the user posted a question on the site; if the user has never posted a question, the value should be null

Note that questions and answers posted after January 31, 2019, should still be included in the results.  And, all users who joined the site in January 2019 should be included (even if they have never posted a question or provided an answer).

The query from the previous question should be a nice starting point to answering this question!  You'll need to use the `posts_answers` and `posts_questions` tables.  You'll also need to use the `users` table from the Stack Overflow dataset.  The relevant columns from the `users` table are `id` (the ID of each user) and `creation_date` (when the user joined the Stack Overflow site, in DATETIME format).

In [14]:
# Your code here
three_tables_query = """
                     SELECT u.id AS id,
                         MIN(q.creation_date) AS q_creation_date,
                         MIN(a.creation_date) AS a_creation_date
                     FROM `bigquery-public-data.stackoverflow.users` AS u
                         FULL JOIN `bigquery-public-data.stackoverflow.posts_answers` AS a
                             ON u.id = a.owner_user_id
                         FULL JOIN `bigquery-public-data.stackoverflow.posts_questions` AS q
                             ON q.owner_user_id = u.id
                     WHERE u.creation_date >= '2019-01-01' and u.creation_date < '2019-02-01'
                     GROUP BY id
                    """

# Check your answer
q_3.check()

,id,q_creation_date,a_creation_date
0,10998283,NaT,2019-01-31 23:29:45.243000+00:00
1,10928490,2019-01-17 14:27:10.790000+00:00,NaT
2,10858682,NaT,NaT
3,10919287,NaT,NaT
4,10977012,NaT,NaT


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [11]:
# Lines below will give you a hint or solution code
q_3.hint()
#q_3.solution()

<IPython.core.display.Javascript object>

<span style="color:#3366cc">Hint:</span> 
Here's a partially completed query.  You need only fill in the appropriate JOINs!
```
three_tables_query = """
                     SELECT u.id AS id,
                         MIN(q.creation_date) AS q_creation_date,
                         MIN(a.creation_date) AS a_creation_date
                     FROM `bigquery-public-data.stackoverflow.users` AS u
                         ____ `bigquery-public-data.stackoverflow.posts_answers` AS a
                             ON u.id = a.owner_user_id
                         ____ `bigquery-public-data.stackoverflow.posts_questions` AS q
                             ON q.owner_user_id = u.id
                     WHERE u.creation_date >= '2019-01-01' and u.creation_date < '2019-02-01'
                     GROUP BY id
                    """
```


### 4) How many distinct users posted on January 1, 2019?

In the code cell below, write a query that returns a table with a single column:
- `owner_user_id` - the IDs of all users who posted at least one question or answer on January 1, 2019.  Each user ID should appear at most once.

In the `posts_questions` (and `posts_answers`) tables, you can get the ID of the original poster from the `owner_user_id` column.  Likewise, the date of the original posting can be found in the `creation_date` column.  

In order for your answer to be marked correct, your query must use a **UNION**.

In [21]:
# Your code here
all_users_query = """
                SELECT DISTINCT(q.owner_user_id) AS owner_user_id,
                FROM `bigquery-public-data.stackoverflow.posts_questions` AS q
                WHERE EXTRACT(DATE FROM q.creation_date) = '2019-01-01'
                UNION DISTINCT
                SELECT DISTINCT(a.owner_user_id) AS owner_user_id
                FROM `bigquery-public-data.stackoverflow.posts_answers` AS a
                WHERE EXTRACT(DATE FROM a.creation_date) = '2019-01-01'
                """

# Check your answer
q_4.check()

<IPython.core.display.Javascript object>

<span style="color:#cc3333">Incorrect:</span> You don't have a valid query yet.  Try again.

In [18]:
# Lines below will give you a hint or solution code
q_4.hint()
q_4.solution()

<IPython.core.display.Javascript object>

<span style="color:#3366cc">Hint:</span> Begin by writing two separate queries: one to get the user IDs corresponding to questions, and another corresponding to answers.  Then, take the union of the two columns.

<IPython.core.display.Javascript object>

<span style="color:#33cc99">Solution:</span> 
```python

all_users_query = """
                  SELECT q.owner_user_id 
                  FROM `bigquery-public-data.stackoverflow.posts_questions` AS q
                  WHERE EXTRACT(DATE FROM q.creation_date) = '2019-01-01'
                  UNION DISTINCT
                  SELECT a.owner_user_id
                  FROM `bigquery-public-data.stackoverflow.posts_answers` AS a
                  WHERE EXTRACT(DATE FROM a.creation_date) = '2019-01-01'
                  """

```

# Keep going

Learn how to use **[analytic functions](https://www.kaggle.com/alexisbcook/analytic-functions)** to perform complex calculations with minimal SQL code.

---




*Have questions or comments? Visit the [Learn Discussion forum](https://www.kaggle.com/learn-forum/161315) to chat with other Learners.*